In [4]:
import pymupdf4llm

path = "/Volumes/WD Green/dev/git/pdf_ocr/pdf-ocr/inputs/2857439.pdf"

md_text = pymupdf4llm.to_markdown(path)
print(md_text)

# Shipping Stem Report

Date Generated: 15/09/2025



Port Ship Name Ref # Exporter Commodity Quantity(tonnes) NominationReceivedDate of NominationAcceptedDate of ETA ETB ETS Load Status



Ship Name Ref #



Date of
Nomination



Accepted



10/07/2025 10/07/2025 06/08/2025

Newcastle ADAGIO NT25084 ARROW COMMODITIES Wheat 26,914
11:45 AM 2:25 PM 8:06 AM



10/07/2025
ADAGIO NT25084 ARROW COMMODITIES Wheat 26,914



10/07/2025 10/07/2025 06/08/2025 06/08/2025 09/08/2025

ARROW COMMODITIES Wheat 26,914 Completed

11:45 AM 2:25 PM 8:06 AM 8:06 AM 11:15 PM



10/07/2025 06/08/2025 06/08/2025 09/08/2025

2:25 PM 8:06 AM 8:06 AM 11:15 PM



2:25 PM



8:06 AM



09/08/2025



10/07/2025 10/07/2025 17/08/2025

Newcastle QC ISABELLA NT25082 QUBE GRAINS Wheat 20,333
7:59 AM 5:40 PM 4:06 PM



10/07/2025
QC ISABELLA NT25082 QUBE GRAINS Wheat 20,333



10/07/2025 10/07/2025 17/08/2025 17/08/2025 19/08/2025

QUBE GRAINS Wheat 20,333 Completed

7:59 AM 5:40 PM 4:06 PM 4:06 PM 11:55 AM



10/07/20

In [6]:
import fitz  # PyMuPDF

def reconstruct_markdown(pdf_path):
    doc = fitz.open(pdf_path)
    full_md = []

    for page in doc:
        # Récupère les blocs de texte triés (haut-gauche vers bas-droite)
        # 'sort=True' gère souvent déjà l'ordre de lecture naturel
        blocks = page.get_text("blocks", sort=True)
        
        for b in blocks:
            x0, y0, x1, y1, text, block_no, block_type = b
            
            if block_type == 0:  # C'est un bloc de texte
                clean_text = text.strip()
                if not clean_text: continue
                
                # Exemple de logique de style simple :
                # Si le bloc est très à gauche, c'est peut-être un titre ou une puce
                if x0 < 100 and len(clean_text) < 50:
                    full_md.append(f"### {clean_text}\n")
                else:
                    full_md.append(f"{clean_text}\n")
        
        full_md.append("\n---\n") # Séparateur de page
    
    return "\n".join(full_md)

md_output = reconstruct_markdown(path)
print(md_output)

Shipping Stem Report

Date Generated: 15/09/2025

Ref #
Ship Name

Date of
Nomination

Received
Port
ETA
Exporter
Quantity
(tonnes)
Commodity
Load Status
ETB
ETS
Date of
Nomination

Accepted

09/08/2025

NT25084
ADAGIO
10/07/2025

### 11:45 AM
Newcastle
06/08/2025

8:06 AM
ARROW COMMODITIES
26,914
Wheat
Completed
06/08/2025

11:15 PM
10/07/2025

2:25 PM

8:06 AM

19/08/2025

### NT25082
QC ISABELLA
10/07/2025

### 7:59 AM
Newcastle
17/08/2025

4:06 PM
QUBE GRAINS
20,333
Wheat
Completed
17/08/2025

11:55 AM
10/07/2025

5:40 PM

4:06 PM

23/08/2025

### NT25085
BRIGHTEN TRADER
01/08/2025

### 7:06 PM
Newcastle
20/08/2024

8:00 AM
ARROW COMMODITIES
33,000
Wheat
Completed
20/08/2024

5:30 PM
04/08/2025

10:39 AM

8:00 AM

31/08/2025

### NT25083
ARUNA NAZIK
25/07/2025

### 12:59 PM
Newcastle
28/08/2025

6:35 AM
CHS BROADBENT
55,000
Wheat
Completed
28/08/2025

11:12 AM
25/07/2025

3:09 PM

6:35 AM

11/09/2025

### NT25086
GREAT FORTUNE
04/08/2025

### 7:26 AM
Newcastle
03/09/2025

1:06 PM
Q

In [ ]:
from pdf_ocr import pdf_to_spatial_text

output = pdf_to_spatial_text("inputs/2857439.pdf")
print(output)

In [ ]:
import os

for fname in sorted(os.listdir("inputs")):
    if not fname.endswith(".pdf"):
        continue
    path = os.path.join("inputs", fname)
    text = pdf_to_spatial_text(path)
    lines = text.split("\n")
    max_width = max((len(l) for l in lines), default=0)
    print(f"{fname:50s}  lines={len(lines):4d}  max_width={max_width:4d}")

## Compressed Spatial Text

`compress_spatial_text()` produces a token-efficient representation by classifying page regions (tables, headings, text blocks, key-value pairs) and rendering them as markdown tables and flowing text instead of whitespace-heavy grids.

In [ ]:
from pdf_ocr import compress_spatial_text, pdf_to_spatial_text

pdf = "inputs/2857439.pdf"

spatial = pdf_to_spatial_text(pdf)
compressed = compress_spatial_text(pdf)

print(compressed)
print(f"\n--- Compression: {len(spatial)} chars → {len(compressed)} chars ({(1 - len(compressed)/len(spatial))*100:.0f}% reduction)")

In [ ]:
import os

print(f"{'File':<50} {'Spatial':>8} {'Compressed':>10} {'Reduction':>10}")
print("-" * 82)

for fname in sorted(os.listdir("inputs")):
    if not fname.endswith(".pdf"):
        continue
    path = os.path.join("inputs", fname)
    s = pdf_to_spatial_text(path)
    c = compress_spatial_text(path)
    reduction = (1 - len(c) / len(s)) * 100 if len(s) > 0 else 0
    print(f"{fname:<50} {len(s):>8} {len(c):>10} {reduction:>9.0f}%")

## Table Interpretation

`interpret_table()` takes compressed text and a **canonical schema** describing the columns your application expects, then uses an LLM pipeline to extract structured records.

The schema maps inconsistent PDF column names (e.g. "Ship Name", "Vessel", "Vessel Name") to stable canonical names via aliases. Two modes are available:

- **2-step** (`interpret_table`) — parse table structure first (GPT-5), then map to schema (GPT-5-mini). More reliable on complex tables.
- **Single-shot** (`interpret_table_single_shot`) — one LLM call. Faster for simple flat tables.

In [ ]:
from pdf_ocr import (
    compress_spatial_text,
    interpret_table,
    interpret_table_single_shot,
    CanonicalSchema,
    ColumnDef,
    to_records,
)

# Define the canonical schema as a plain dict (e.g. loaded from a JSON file).
# CanonicalSchema.from_dict() converts it into the typed dataclass.
#
# Note: "port" has no aliases — it will be inferred from context (section headers,
# document title, or repeated contextual values) rather than matched to a column name.
schema_dict = {
    "description": "Shipping stem vessel loading records",
    "columns": [
        {"name": "port",            "type": "string", "description": "Loading port name — may appear as a section header or document-level context rather than a table column", "aliases": []},
        {"name": "vessel_name",     "type": "string", "description": "Name of the vessel",             "aliases": ["Ship Name", "Vessel"]},
        {"name": "ref",             "type": "string", "description": "Reference number",               "aliases": ["Ref #", "Reference"]},
        {"name": "exporter",        "type": "string", "description": "Exporting company",              "aliases": ["Exporter"]},
        {"name": "commodity",       "type": "string", "description": "Type of commodity",              "aliases": ["Commodity"]},
        {"name": "quantity_tonnes", "type": "int",    "description": "Quantity in metric tonnes",      "aliases": ["Quantity", "Quantity(tonnes)", "Total"]},
        {"name": "eta",             "type": "string", "description": "Estimated time of arrival",      "aliases": ["ETA", "Date ETA of Ship"]},
        {"name": "etb",             "type": "string", "description": "Estimated time of berthing",     "aliases": ["ETB"]},
        {"name": "ets",             "type": "string", "description": "Estimated time of sailing",      "aliases": ["ETS"]},
        {"name": "status",          "type": "string", "description": "Loading status",                 "aliases": ["Status", "Load Status"]},
    ],
}

schema = CanonicalSchema.from_dict(schema_dict)

print(f"Schema: {schema.description}")
print(f"Columns ({len(schema.columns)}):")
for col in schema.columns:
    print(f"  {col.name:20s}  {col.type:6s}  aliases={col.aliases}")

In [ ]:
# Compress the shipping stem PDF, then run the 2-step interpret pipeline.
# Step 1 (AnalyzeAndParseTable) identifies structure and parses rows.
# Step 2 (MapToCanonicalSchema) maps columns to our canonical names.

compressed = compress_spatial_text("inputs/2857439.pdf")
result = interpret_table(compressed, schema, model="openai/gpt-4o")

print(f"Records:           {len(result.records)}")
print(f"Unmapped columns:  {result.unmapped_columns}")
if result.mapping_notes:
    print(f"Mapping notes:     {result.mapping_notes}")

print("\n--- Extracted records ---\n")
for i, rec in enumerate(to_records(result), 1):
    print(f"[{i}] {rec}")

In [ ]:
# Interpretation metadata: model used, per-field mapping rationale, detected sections.
# This shows HOW each canonical column was resolved — direct match, alias, or context inference.

meta = result.metadata
print(f"Model: {meta.model}")
print(f"Sections detected: {meta.sections_detected}")
print(f"\nField mappings ({len(meta.field_mappings)}):\n")
for fm in meta.field_mappings:
    print(f"  {fm.column_name:20s}  confidence={fm.confidence.value:6s}  source={fm.source}")
    print(f"  {'':20s}  rationale: {fm.rationale}")
    print()

In [ ]:
# Compare: single-shot mode does both steps in one LLM call.
# Faster (1 round-trip instead of 2), but may be less accurate on complex tables.

result_ss = interpret_table_single_shot(compressed, schema, model="openai/gpt-4o")

print(f"Records (single-shot):  {len(result_ss.records)}")
print(f"Unmapped columns:       {result_ss.unmapped_columns}")

print("\n--- Single-shot records ---\n")
for i, rec in enumerate(to_records(result_ss), 1):
    print(f"[{i}] {rec}")